# Lab 03 - Spark

## Information

| Student name       | ID       |
|--------------------|----------|
| Nguyễn Thiên Phúc  | 20127681 |
| Lê Nguyễn Nhật Phú | 20127275 |
| Võ Hiền Hải Thuận  | 20127344 |


## Note

**`Restart Kernel` and `Run all cells` before submitting on Moodle or your work will be counted as 0!**


## Assignment

### Import libraries

In [1]:
import findspark 
findspark.init()
findspark.find()
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import scipy
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("").getOrCreate()
from pyspark.python.pyspark.shell import spark

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.4.0
      /_/

Using Python version 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022 23:13:41)
Spark context Web UI available at http://Phuc-Nguyen:4040
Spark context available as 'sc' (master = local[*], app id = local-1682397379570).
SparkSession available as 'spark'.


### Requirement 1

**Problem 1**: Decision Tree

In [2]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")
data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Height'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)
data.show(5)

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours|MOA|label| ROA|distance| BMI|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| 11|                26| 

In [3]:
assem = VectorAssembler(inputCols=["MOA", "distance", "ROA", "BMI"], outputCol='features')
data = assem.transform(data)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [4]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0| [3.0,11.0,0.0,29.0]|
|       1.0|         1.0|[8.0,11.0,19.0,29.0]|
|       1.0|         1.0|[3.0,11.0,22.0,29.0]|
|       1.0|         1.0|[5.0,11.0,22.0,29.0]|
|       1.0|         1.0|[8.0,11.0,23.0,29.0]|
+----------+------------+--------------------+
only showing top 5 rows



In [5]:
confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

treeModel = model.stages[2]
# summary only
print(treeModel)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))

print("The precision score for Decision Tree Model is: " + str(precision))

print("The recall score for Decision Tree Model is: " + str(recall))

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_729dbeed43e3, depth=5, numNodes=27, numClasses=14, numFeatures=4
Decision Tree - Test Accuracy = 0.817352
Decision Tree - Test Error = 0.182648
The Confusion Matrix for Decision Tree Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Decision Tree Model is: 0.02972972972972973
The recall score for Decision Tree Model is: 0.02972972972972973


**Problem 2**: Naive Bayes

In [6]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")
data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Seasons'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)

assem = VectorAssembler(inputCols=["MOA", "distance", "ROA", "BMI"], outputCol='features')

data = assem.transform(data)
# Split the data into train and test
splits = data.randomSplit([0.7, 0.3], 1000)
train = splits[0]
test = splits[1]

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")

y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()


accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')


print("Naive Bayes - Test set accuracy = " + str(accuracy))

print("Naive Bayes - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Naive Bayes Model is :\n" + str(confusionmatrix))

print("The precision score for Naive Bayes Model is: " + str(precision))

print("The recall score for Naive Bayes Model is: " + str(recall))

Naive Bayes - Test set accuracy = 0.061224489795918366
Naive Bayes - Test Error = 0.938776
The Confusion Matrix for Naive Bayes Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Naive Bayes Model is: 0.02972972972972973
The recall score for Naive Bayes Model is: 0.02972972972972973


**Problem 3**: Random Forest

In [7]:
# your code goes here
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")
data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Transportation expense'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.

assem = VectorAssembler(inputCols=["MOA", "distance", "ROA", "BMI"], outputCol='features')

data = assem.transform(data)

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))


+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|         235.0|235.0| [3.0,11.0,0.0,29.0]|
|         235.0|235.0|[6.0,11.0,13.0,29.0]|
|         235.0|235.0|[7.0,11.0,22.0,29.0]|
|         235.0|235.0|[8.0,11.0,23.0,29.0]|
|         235.0|235.0|[11.0,11.0,25.0,2...|
+--------------+-----+--------------------+
only showing top 5 rows

RandomForestClassificationModel: uid=RandomForestClassifier_e35b4fe9b784, numTrees=10, numClasses=24, numFeatures=4
Random Forest - Test Accuracy = 0.872807
Random Forest - Test Error = 0.127193
The Confusion Matrix for Random Forest Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Random Forest Model is: 0.02972972972972973
The recall score for Random Forest Model is: 0.02972972972972973


### Requirement 2

**Dataset**:

**Dataset 1: Iris Dataset**

Description: It includes three iris species with 50 samples each as well as some properties about each flower. One flower species is linearly separable from the other two, but the other two are not linearly separable from each other.

Link  [Iris dataset](https://www.kaggle.com/datasets/uciml/iris)


**Dataset 2: Red Wine Dataset**

Description: This datasets is related to red variants of the Portuguese "Vinho Verde" wine. For more details, consult the reference [Cortez et al., 2009]. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

Link  [Red wine dataset](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009)

**Implementation**

**On Iris Dataset**

a. Random forest on Iris dataset

In [8]:
# your code goes here
from pyspark.python.pyspark.shell import spark
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [19]:
# your code goes here
data = spark.read.load("Iris.csv", format="csv", header=True, delimiter=",")

data = data.withColumn("SepalLength", data["SepalLengthCm"] - 0).withColumn("SepalWidth", data['SepalWidthCm'] - 0). \
    withColumn("PetalLength", data["PetalLengthCm"] - 0). \
    withColumn("PetalWidth", data["PetalWidthCm"] - 0)
data.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|SepalLength|SepalWidth|PetalLength|PetalWidth|
+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|        5.1|       3.5|        1.4|       0.2|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|        4.9|       3.0|        1.4|       0.2|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|        4.7|       3.2|        1.3|       0.2|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|        4.6|       3.1|        1.5|       0.2|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|        5.0|       3.6|        1.4|       0.2|
+---+-------------+------------+-------------+----------

In [20]:
assem = VectorAssembler(inputCols=["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"], outputCol='features')
data = assem.transform(data)
data.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|SepalLength|SepalWidth|PetalLength|PetalWidth|         features|
+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|        5.1|       3.5|        1.4|       0.2|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|        4.9|       3.0|        1.4|       0.2|[4.9,3.0,1.4,0.2]|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|        4.7|       3.2|        1.3|       0.2|[4.7,3.2,1.3,0.2]|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|        4.6|       3.1|        1.5|       0.2|[4.6,3.1,1.5,0.2]|
|  5|          5.0|         3.6|          1.4|  

In [21]:
#Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="Species", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])


In [22]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [23]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
predictions.show(5)

+---+-------------+------------+-------------+------------+--------------+-----------+----------+-----------+----------+-----------------+------------+-----------------+--------------------+--------------------+----------+--------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|       Species|SepalLength|SepalWidth|PetalLength|PetalWidth|         features|indexedLabel|  indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|
+---+-------------+------------+-------------+------------+--------------+-----------+----------+-----------+----------+-----------------+------------+-----------------+--------------------+--------------------+----------+--------------+
|102|          5.8|         2.7|          5.1|         1.9|Iris-virginica|        5.8|       2.7|        5.1|       1.9|[5.8,2.7,5.1,1.9]|         2.0|[5.8,2.7,5.1,1.9]|[0.0,4.1428571428...|[0.0,0.4142857142...|       2.0|Iris-virginica|
|108|          7.3|         2.9|          6.3|  

In [24]:
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

In [25]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))

RandomForestClassificationModel: uid=RandomForestClassifier_4df80740591c, numTrees=10, numClasses=3, numFeatures=4
Random Forest - Test Accuracy = 0.979167
Random Forest - Test Error = 0.0208333
The Confusion Matrix for Random Forest Model is :
[[14  0  0]
 [ 0 15  1]
 [ 0  0 18]]
The precision score for Random Forest Model is: 0.9791666666666666
The recall score for Random Forest Model is: 0.9791666666666666


b. Decision Tree on Iris dataset

In [26]:
#Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="Species", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [27]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

In [28]:
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

In [29]:
confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

treeModel = model.stages[2]
# summary only
print(treeModel)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))

print("The precision score for Decision Tree Model is: " + str(precision))

print("The recall score for Decision Tree Model is: " + str(recall))

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b81c98937932, depth=4, numNodes=13, numClasses=3, numFeatures=4
Decision Tree - Test Accuracy = 0.924528
Decision Tree - Test Error = 0.0754717
The Confusion Matrix for Decision Tree Model is :
[[18  0  0]
 [ 0 17  3]
 [ 0  1 14]]
The precision score for Decision Tree Model is: 0.9245283018867925
The recall score for Decision Tree Model is: 0.9245283018867925


**On Red Wine Dataset**

In [30]:
data = spark.read.load("winequality-red.csv", format="csv", header=True, delimiter=";")
data = data.withColumn("fixed acidity", data["fixed acidity"] - 0).withColumn("volatile acidity", data['volatile acidity'] - 0). \
    withColumn("citric acid", data["citric acid"] - 0). \
    withColumn("residual sugar", data["residual sugar"] - 0). \
    withColumn("chlorides", data["chlorides"] - 0). \
    withColumn("free sulfur dioxide", data["free sulfur dioxide"] - 0). \
    withColumn("total sulfur dioxide", data["total sulfur dioxide"] - 0). \
    withColumn("density", data["density"] - 0). \
    withColumn("pH", data["pH"] - 0). \
    withColumn("sulphates", data["sulphates"] - 0). \
    withColumn("alcohol", data["alcohol"] - 0). \
    withColumn("quality", data["quality"] - 0)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    5.0|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [31]:
assem = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar",
                                    "chlorides", "free sulfur dioxide","total sulfur dioxide", "density", 
                                    "pH", "sulphates", "alcohol"], outputCol='features')
data = assem.transform(data)
data.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)
 |-- features: vector (nullable = true)



In [32]:
#Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="quality", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=12).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [33]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="quality", featuresCol="features")

# Fit the model
lrModel = lr.fit(trainingData)

In [34]:
test_stats = lrModel.evaluate(testData)
print(f"RMSE: {test_stats.rootMeanSquaredError}")
print(f"R2: {test_stats.r2}")
print(f"MSE: {test_stats.meanSquaredError}")

RMSE: 0.6295361470266028
R2: 0.3432513421012249
MSE: 0.39631576041310035
